In [1]:
# save as convert_fits_to_png.py
#!/usr/bin/env python3
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2  # OpenCV for resizing

# Define input FITS files and output directory
fits_files = [
    '/users/vutomi/Source_Detection/abell_209.fits'
]
output_dir = '/users/vutomi/Source_Detection/YOLOs/Data/images'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

def fits_to_png(fits_file, output_dir):
    # Generate output PNG and 2D FITS filenames
    base_name = os.path.splitext(os.path.basename(fits_file))[0]
    output_file = os.path.join(output_dir, f"{base_name}.png")
    output_2d_fits = os.path.join(output_dir, f"{base_name}_2d.fits")

    # Load the FITS file
    try:
        with fits.open(fits_file) as hdul:
            data = hdul[0].data  # Original image
            header = hdul[0].header
    except FileNotFoundError:
        print(f"Error: FITS file not found at {fits_file}. Please check the path.")
        return False

    # Collapse multi-dimensional data (e.g., frequency/stokes axes)
    if data.ndim > 2:
        data = np.nanmean(data, axis=(0, 1) if data.ndim == 4 else 0)
        # Update header for 2D FITS
        header['NAXIS'] = 2
        for key in ['NAXIS3', 'NAXIS4', 'CRVAL3', 'CRPIX3', 'CDELT3', 'CTYPE3', 'CRVAL4', 'CRPIX4', 'CDELT4', 'CTYPE4']:
            header.remove(key, ignore_missing=True)

    # Save 2D FITS for WCS usage in labeling
    try:
        fits.writeto(output_2d_fits, data, header, overwrite=True)
        print(f"2D FITS saved successfully at {output_2d_fits}")
    except Exception as e:
        print(f"Error saving 2D FITS for {fits_file}: {e}")
        return False

    # Normalize the data to 0-255 for PNG conversion
    data = np.nan_to_num(data, nan=0.0)  # Replace NaNs
    data = np.log1p(data)  # Log-scale to enhance faint sources
    data_min, data_max = np.percentile(data, (1, 99))  # Clip extreme values
    data = np.clip(data, data_min, data_max)
    data = (data - data_min) / (data_max - data_min) * 255  # Scale to 0-255
    data = data.astype(np.uint8)

    # Resize to YOLO-compatible dimensions (nearest multiple of 32)
    target_size = (1024, 1024)  # Adjust as needed
    data_resized = cv2.resize(data, target_size, interpolation=cv2.INTER_AREA)
    # Save as PNG with error handling
    try:
        plt.imsave(output_file, data_resized, cmap='gray')
        print(f"PNG saved successfully at {output_file}")
    except Exception as e:
        print(f"Error saving PNG for {fits_file}: {e}")
        return False

    # Save 2D FITS for WCS usage in labeling
    try:
        fits.writeto(output_2d_fits, data, header, overwrite=True)
        print(f"2D FITS saved successfully at {output_2d_fits}")
    except Exception as e:
        print(f"Error saving 2D FITS for {fits_file}: {e}")
        return False

    # Verify the file exists
    if os.path.exists(output_file):
        print(f"File verified on disk: {output_file}")
        return True
    else:
        print(f"File not found on disk: {output_file}. Something went wrong.")
        return False

# Process each FITS file
for fits_file in fits_files:
    fits_to_png(fits_file, output_dir)

2D FITS saved successfully at /users/vutomi/Source_Detection/YOLOs/Data/images/abell_209_2d.fits
PNG saved successfully at /users/vutomi/Source_Detection/YOLOs/Data/images/abell_209.png
2D FITS saved successfully at /users/vutomi/Source_Detection/YOLOs/Data/images/abell_209_2d.fits
File verified on disk: /users/vutomi/Source_Detection/YOLOs/Data/images/abell_209.png
